In [1]:
import math
import numpy as np
import wandb

import torch
import torch_geometric
from torch_geometric.data import Data

from gnn_architectures import MyGnn

import gnn_io as gio
import gnn_architectures as garch

import pprint

## 1. Define model and parameters

In [2]:
wandb.login()

# Define parameters 

num_epochs = 1000
project_name = 'finetuning'
train_ratio = 0.8

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

early_stopping = gio.EarlyStopping(patience=10, verbose=True)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: enatterer (tum-traffic-engineering). Use `wandb login --relogin` to force relogin


## 2. Load data

In [3]:
# Load the list of dictionaries
data_dict_list = torch.load('../data/dataset_1pm_0-1382.pt')
# torch.load('../data/dataset_1pm_0-1382_with_more_infos.pt') if model_is_basic else 

# Reconstruct the Data objects
datalist = [Data(x=d['x'], edge_index=d['edge_index'], pos=d['pos'], y=d['y']) for d in data_dict_list]

# # Apply normalization to your dataset
dataset_normalized = gio.normalize_dataset(datalist)

baseline_error = gio.compute_baseline_error(datalist)
print(f'Baseline error: {baseline_error}')

Baseline error: 0.005565311759710312


## 3. Split into train and test set

In [4]:
# n_steps_per_epoch = math.ceil(len(train_dl.dataset) / config.batch_size)

## 4. Train the model

We first find a good model for one batch. 

In [5]:
def train(model, config=None, loss_fct=None, optimizer=None, train_dl=None, valid_dl=None):
    for epoch in range(config.epochs):
        model.train()
        for idx, data in enumerate(train_dl):
            input_node_features, targets = data.x.to(device), data.y.to(device)
            optimizer.zero_grad()

            # Forward pass
            predicted = model(data)
            train_loss = loss_fct(predicted, targets)
            
            # Backward pass
            train_loss.backward()
            optimizer.step()
            
            wandb.log({"train_loss": train_loss.item(), "epoch": epoch, "step": idx})
            # print(f"epoch: {epoch}, step: {idx}, loss: {train_loss.item()}")
        
        val_loss = garch.validate_model_pos_features(model, valid_dl, loss_fct, device)
        print(f"epoch: {epoch}, validation loss: {val_loss}")
        wandb.log({"loss": val_loss, "epoch": epoch})
            
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print("Early stopping triggered. Stopping training.")
            break
    
    print("Best validation loss: ", val_loss)
    wandb.summary["val_loss"] = val_loss
    wandb.finish()

In [6]:
# import itertools

# batch_size_range = [16, 24, 32]
# output_layer_range = ['gat', 'gcn']

# hidden_size_range = [16, 32, 64]
# gat_layers_range = [0, 1, 2, 3]
# gcn_layers_range = [0, 1, 2, 3]
# lr_range = [0.001, 0.0001]


# for batch_size in batch_size_range:
#     for lr in lr_range:
#         for output_layer_parameter in output_layer_range:
#             for hidden_size_parameter in hidden_size_range:
#                 for gat_layer_parameter in gat_layers_range:
#                     for gcn_layer_parameter in gcn_layers_range:
#                         train_dl = gio.create_dataloader(dataset=dataset_normalized, is_train=True, batch_size=batch_size, train_ratio=train_ratio)
#                         valid_dl = gio.create_dataloader(dataset=dataset_normalized, is_train=False, batch_size=batch_size, train_ratio=train_ratio)
#                         wandb.init(
#                                 project=project_name,
#                                 config={
#                                     "epochs": num_epochs,
#                                     "batch_size": batch_size,
#                                     "lr": lr,
#                                     'early_stopping_patience': 10,
#                                     # "dropout": 0.15,
#                                     })
#                         config = wandb.config
                        
#                         print("output_layer: ", output_layer_parameter)
#                         print("hidden_size: ", hidden_size_parameter)
#                         print("gat_layers: ", gat_layer_parameter)
#                         print("gcn_layers: ", gcn_layer_parameter)
#                         gnn_instance = MyGnn(in_channels=3, out_channels=1, hidden_size=hidden_size_parameter, gat_layers=gat_layer_parameter, gcn_layers=gcn_layer_parameter, output_layer=output_layer_parameter)
#                         model = gnn_instance.to(device)
#                         train(model, config=config, loss_fct=torch.nn.MSELoss(), optimizer=torch.optim.Adam(model.parameters(), lr=lr), train_dl = train_dl, valid_dl = valid_dl)

In [7]:
import itertools

batch_size_range = [16, 24, 32]
output_layer_range = ['gat', 'gcn']

hidden_size_range = [16, 32, 64]
gat_layers_range = [0, 1, 2]
gcn_layers_range = [0, 1, 2]
lr_range = [0.001, 0.0001]

# Create a list to hold all configurations
configurations = list(itertools.product(batch_size_range, lr_range, output_layer_range, hidden_size_range, gat_layers_range, gcn_layers_range))

counter = 0
# Open the text file for writing
with open('model_performance.txt', 'w') as f:
    for config in configurations:
        counter +=1
        print(f"Configuration {counter}/{len(configurations)}")
        batch_size, lr, output_layer_parameter, hidden_size_parameter, gat_layer_parameter, gcn_layer_parameter = config

        train_dl = gio.create_dataloader(dataset=dataset_normalized, is_train=True, batch_size=batch_size, train_ratio=train_ratio)
        valid_dl = gio.create_dataloader(dataset=dataset_normalized, is_train=False, batch_size=batch_size, train_ratio=train_ratio)
        
        wandb.init(
            project=project_name,
            config={
                "epochs": num_epochs,
                "batch_size": batch_size,
                "lr": lr,
                'early_stopping_patience': 10,
                # "dropout": 0.15,
            }
        )
        config = wandb.config
        
        print("output_layer: ", output_layer_parameter)
        print("hidden_size: ", hidden_size_parameter)
        print("gat_layers: ", gat_layer_parameter)
        print("gcn_layers: ", gcn_layer_parameter)
        
        gnn_instance = MyGnn(in_channels=3, out_channels=1, hidden_size=hidden_size_parameter, gat_layers=gat_layer_parameter, gcn_layers=gcn_layer_parameter, output_layer=output_layer_parameter)
        model = gnn_instance.to(device)
        
        best_val_loss, best_epoch = train(model, config=config, loss_fct=torch.nn.MSELoss(), optimizer=torch.optim.Adam(model.parameters(), lr=lr), train_dl=train_dl, valid_dl=valid_dl)
        
        # Write the configuration and the best loss to the text file
        f.write(f"Configuration: batch_size={batch_size}, lr={lr}, output_layer={output_layer_parameter}, hidden_size={hidden_size_parameter}, gat_layers={gat_layer_parameter}, gcn_layers={gcn_layer_parameter}\n")
        f.write(f"Best Validation Loss: {best_val_loss}, Best Epoch: {best_epoch}\n\n")

Configuration 1/324
Total dataset length: 1382
Training subset length: 1104
Total dataset length: 1382
Validation subset length: 272


output_layer:  gat
hidden_size:  16
gat_layers:  0
gcn_layers:  0
Model initialized
MyGnn(
  (pointLayer): PointNetConv(local_nn=Sequential(
    (0): Linear(in_features=3, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=16, bias=True)
  ), global_nn=Sequential(
    (0): Linear(in_features=16, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=16, bias=True)
  ))
  (output_layer): GATConv(16, 1, heads=1)
)


/Users/elenanatterer/anaconda3/envs/ml_env/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([499456, 1])) that is different to the input size (torch.Size([499456, 16])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch: 0, validation loss: 0.005564481156932957


In [ ]:
len(configurations)

324